## Biodiversity Intactness Index for Phoneix, Arizona from 2017 to 2020

Taking a brief look at changes in the biodiversity intactness index (BII) for Phoneinx, Arzona, by pulling desired data and items frokm a catalog

In [1]:
import pandas as pd
import planetary_computer
import pystac_client
import rich.table
from geogif import gif

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
bbox_of_interest = [-112.826843, 32.974108, -111.184387, 33.863574]
BII_Phoneix = catalog.search(collections=["io-biodiversity"], bbox=bbox_of_interest)

items = list(BII_Phoneix.items())
for item in items:
    print(item)

<Item id=bii_2020_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2019_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2018_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2017_34.74464974521749_-115.38597824385106_cog>


Searching the catalog for 

In [ ]:
# Creating our bounding box to analyze the 

In [4]:
# Temporal range of interest
time_range = "2017-01-01/2020-01-01"

# Catalog search 
search= catalog.search(
    collections= ['io-biodiversity'],
    intersects= bbox,
    datetime= time_range,
)
search

NameError: name 'bbox' is not defined